In [9]:
from PIL import Image,ImageDraw,ImageFont
from ultralytics import YOLO
from os import walk
import easyocr
import numpy as np
import xlsxwriter
import os

In [8]:

model = YOLO('./weights/bestV2.pt')  # load a custom model


In [10]:
myFont = ImageFont.truetype('./Roboto-Bold.ttf', 30)
reader = easyocr.Reader(['en'])
mypath="./Image_files/"
filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file

output_path="./output-marked_img"
if not os.path.exists(output_path):
    os.makedirs(output_path)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [12]:
for filename in filenames:
    img_path=mypath+"/"+filename
    img = Image.open(img_path).convert('RGB')

    #img=is_vertically_flipped(img)
    #img.save(img_path)

    results = model.predict(img, conf=0.30)

    text_result = reader.readtext(img_path, detail = 0)

    ref_no=""
    for txts in text_result:
        if(txts.__contains__("Ref#")):
            ref_no=txts.split("#")[1].split("/")[0]

    i=0
    for det in results[0].boxes.xyxy:
        x1, y1, x2, y2 = det
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        draw = ImageDraw.Draw(img)
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
        draw2 = ImageDraw.Draw(img)
        draw2.text((x1+10, y1+10),ref_no, font=myFont,fill=(255, 0, 0),outline=(0,0,0))
        i=i+1
    draw = ImageDraw.Draw(img)
    if results[0].boxes.xyxy.size()[0]>=2:
        draw2.text((10, 10),"Picture with two ink stamps", font=myFont,fill=(255, 0, 0),outline=(0,0,0))
    else:
        draw2.text((10, 10),"Picture with less than two ink stamps", font=myFont,fill=(255, 0, 0),outline=(0,0,0))

    seal_present="Yes"
    if(results[0].boxes.xyxy.size()[0]>=2):
        seal_present="Yes"
    else:
        seal_present="No"


    img.save(output_path+"/"+ref_no+filename)

    img.show()


0: 512x640 3 squaresealss, 1732.6ms
Speed: 11.9ms preprocess, 1732.6ms inference, 15.7ms postprocess per image at shape (1, 3, 512, 640)

0: 448x640 3 squaresealss, 1764.6ms
Speed: 3.6ms preprocess, 1764.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 640x480 (no detections), 1701.5ms
Speed: 49.5ms preprocess, 1701.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 512x640 1 roundseals, 1851.7ms
Speed: 8.9ms preprocess, 1851.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

0: 640x480 2 rectanglesealss, 1 squareseals, 1809.8ms
Speed: 23.5ms preprocess, 1809.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x448 1 rectangleseals, 4 roundsealss, 1 squareseals, 1772.0ms
Speed: 17.2ms preprocess, 1772.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 448)

0: 480x640 2 rectanglesealss, 1905.7ms
Speed: 0.0ms preprocess, 1905.7ms inference, 0.0ms postprocess per image at shape (1, 3, 48